In [ ]:
import re

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def input_process(text):
    pattern = r"p=(\d+),(\d+) v=(-?\d+),(-?\d+)"
    match = re.search(pattern, text)
    out = (
        np.array([int(match.group(1)), int(match.group(2))]),
        np.array([int(match.group(3)), int(match.group(4))]),
    )
    return out

In [ ]:
def is_valid_index(array, index):
    return (index >= 0).all() and (index < array.shape).all()


def add_to_region(grid, visited, idx, region):
    if is_valid_index(grid, idx) and not visited[*idx] and grid[*idx]:
        region.add((int(idx[0]), int(idx[1])))
        visited[*idx] = True
        for offset in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
            add_to_region(grid, visited, idx + offset, region)


def get_largest_regions(grid):
    visited = np.zeros_like(grid)
    regions = []
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if grid[i][j] and not visited[i][j]:
                region = set()
                add_to_region(grid, visited, np.array([i, j]), region)
                regions.append(region)
    regions.sort(key=len, reverse=True)
    return regions[0]

In [ ]:
with open("data/day14/input.txt", "r") as file:
    robot_init_raw = file.read()
grid_size = np.array([101, 103])

## Part 1

In [ ]:
class Robot:
    def __init__(self, pos_init, vel, grid_size):
        self.pos_init = pos_init
        self.vel = vel
        self.grid_size = grid_size

        self.pos = self.pos_init

    def __call__(self):
        self.pos += self.vel
        overshoot = np.floor(self.pos / self.grid_size).astype(int)
        self.pos -= overshoot * self.grid_size

    def get_quadrant(self):
        midpoint = ((self.grid_size - 1) / 2).astype(int)
        wn = self.pos < midpoint
        eq_mid = self.pos == midpoint
        if any(eq_mid):
            out = None
        else:
            out = ("N" if wn[1] else "S") + ("W" if wn[0] else "E")
        return out

In [ ]:
robots = [
    Robot(pos_init=x[0], vel=x[1], grid_size=grid_size)
    for x in [input_process(text) for text in robot_init_raw.split("\n")]
]
for _ in range(100):
    for robot in robots:
        robot()
robot_quadrant = [robot.get_quadrant() for robot in robots]

In [ ]:
int(np.prod(np.unique([x for x in robot_quadrant if x], return_counts=True)[1]))

## Part 2

In [ ]:
robots = [
    Robot(pos_init=x[0], vel=x[1], grid_size=grid_size)
    for x in [input_process(text) for text in robot_init_raw.split("\n")]
]

In [ ]:
tree_found = False
iteration = 0
while not tree_found:

    grid = np.zeros((grid_size[1], grid_size[0]), dtype=bool)
    for robot in robots:
        grid[(robot.pos[1], robot.pos[0])] = True
    tree_ind_poss = get_largest_regions(grid)

    if iteration % 250 == 0:
        print(f"-- {iteration} --")

    if (
        np.mean([(robot.pos[1], robot.pos[0]) in tree_ind_poss for robot in robots])
        > 0.25
    ):
        print(f"** {iteration} **")
        plt.imshow(grid)
        plt.show()
        tree_found = True
    else:
        iteration += 1
        for robot in robots:
            robot()